In [70]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from datetime import date
import altair as alt

from statistics import *

file = pd.read_csv("merged_stop_data.csv", dtype = str)
file = file[file["Bus"] != "0"]
file = file[file["Time"].notna()]
file = file[file["Date"].notna()]
file = file[file["Route"].isin(["Gold", "Silver", "Green"])]

SyntaxError: invalid syntax (<ipython-input-70-271f39aec5f6>, line 14)

In [64]:
def calculate_distance(start_stop, destination_stop, hour, day, month):
    stop_filter = file[file["Stop"].isin([start_stop, destination_stop])]
    time_column = [datetime.strptime(f"{date}_{time}", "%m/%d/%Y_%H:%M:%S") for time, date in zip(stop_filter["Time"], stop_filter["Date"])]
    stop_filter = stop_filter.assign(Converted_Time=time_column)
    times = []
    for bus, data in stop_filter.groupby("Bus"):
        previous_stop = ""
        previous_time = 0
        previous_route = ""

        for current_stop, current_time, route in zip(data["Stop"], data["Converted_Time"], data["Route"]):

            correct_stop_order = current_stop == destination_stop and previous_stop == start_stop
            consistant_route = previous_route == route
            correct_hour = current_time.hour == hour
            correct_day = current_time.weekday() == day
            correct_month = current_time.month == month

            if(correct_stop_order and consistant_route and correct_hour and correct_day and correct_month and current_time.year == 2021):
                time_difference = current_time - previous_time
                times.append(round(time_difference.seconds / 60, 2))

            previous_stop = current_stop
            previous_time = current_time
            previous_route = route
    
    filtered_time = [int(t) for t in times if t < 20]
    if(len(filtered_time) == 0):
        return -1, []
    # print(f"Median at hour {hour} on day {day}:", median(filtered_time))
    return median(filtered_time), filtered_time

In [73]:
now = datetime.now()
first_location = "CRI Deck"
second_location = "Lot 5A"
print(calculate_distance(first_location, second_location, now.hour, now.weekday(), now.month))

(11, [15, 12, 10, 13, 11, 8, 14, 10, 3, 13, 12, 0, 8])


In [69]:
dict = {}
for first_stop, first_route in zip(file["Stop"], file["Route"]):
    for second_stop, second_route in zip(file["Stop"], file["Route"]):
        key = f"{first_stop} > {second_stop}"
        if second_stop != first_stop and not key in dict and first_route == second_route:
            distance, times = calculate_distance(first_stop, second_stop, now.hour, now.weekday(), now.month)
            dict[key] = distance

            if times == []:
                print("------------")
            else:
                print(f"{key}: {distance} ({len(times)})")

CRI Deck > Martin Hall: 9.0 (10)
CRI Deck > Lot 5A: 11 (13)
------------
CRI Deck > Lot 6: 9 (11)
CRI Deck > Student Union W: 13.0 (12)
CRI Deck > Grigg Hall E: 4.0 (10)
CRI Deck > Student Union E: 7 (9)
CRI Deck > Student Health N: 8.5 (4)
------------
------------
------------
CRI Deck > Aux Services East: 6 (5)
CRI Deck > Grigg Hall W: 15 (9)
CRI Deck > East Deck 2: 13 (3)
CRI Deck > Athletics Complex W: 5 (1)
------------
CRI Deck > EPIC N: 5.5 (2)
CRI Deck > Fretwell N: 13.5 (8)
------------
CRI Deck > Athletics Complex E: 6 (1)
------------
------------
CRI Deck > Motor Sports: 17.0 (2)
CRI Deck > EPIC S: 16 (3)
------------
------------
------------
------------
------------
------------
------------
------------
Martin Hall > CRI Deck: 11 (7)
Martin Hall > Lot 5A: 2 (15)
------------
Martin Hall > Lot 6: 1.0 (12)
Martin Hall > Student Union W: 5.0 (14)
Martin Hall > Grigg Hall E: 16 (15)
Martin Hall > Student Union E: 19.0 (8)
Martin Hall > Student Health N: 0 (1)
------------


KeyboardInterrupt: 